In [278]:
from lxml import etree
import pandas as pd
from tqdm import tqdm
import os



In [279]:
#setup
path = 'OneDrive_2_19-04-2023'
xml_file= '20230421-100059-ALL CONTRACTS.xml'
excel_crm = 'source_data/templates/2023-04 17 CRM_Contract_Conversion_Template_v0.4.xlsx'
excel_nmd = 'source_data/templates/2023-04-17 NMD_Contract_Conversion_Template_v0.4.xlsx'
sheet_name1 = 'Price Program - Static'
sheet_name2 = 'Price Prog - Static-Tiered'
sheet_name_rem = 'Contract Header'

pricing_crm = 'source_data/pricing/2023-04-13 CRM SalesDeal-CRM-ZVSLSDLDL-2023-04-13.csv'
pricing_nmd = 'source_data/pricing/2023-04-13 NMD SalesDeal-NMD-2023-04-14.csv'
remove = '2023-03-31_PRD_DEL.xlsx'


In [280]:
# Opening excel and xml file
tree = etree.parse(f'{path}/{xml_file}')
root=tree.getroot()

excel_crm1 = pd.read_excel(f'{path}/{excel_crm}', header = 2, sheet_name=f'{sheet_name1}')
excel_crm2 = pd.read_excel(f'{path}/{excel_crm}', header = 2, sheet_name=f'{sheet_name2}')
excel_crm_rem = pd.read_excel(f'{path}/{excel_crm}', header = 2, sheet_name=f'{sheet_name_rem}')
excel_nmd1 = pd.read_excel(f'{path}/{excel_nmd}', header = 2, sheet_name=f'{sheet_name1}')
excel_nmd2 = pd.read_excel(f'{path}/{excel_nmd}', header = 2, sheet_name=f'{sheet_name2}')
excel_nmd_rem = pd.read_excel(f'{path}/{excel_nmd}', header = 2, sheet_name=f'{sheet_name_rem}')
pricing_crm = pd.read_csv(f'{path}/{pricing_crm}')
pricing_nmd = pd.read_csv(f'{path}/{pricing_nmd}')
remove = pd.read_excel(f'{path}/{remove}')
#excel_nmd = pd.read_excel(f'{path}/{excel_nmd}', header = 2, sheet_name=f'{sheet_name}')
#load = pd.read_excel(f'{path}/{load_file}')
#df_log = pd.read_csv(f'{path}/logs.csv', header=None)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(

In [281]:
# contracts to be removed
excel_crm_rem = excel_crm_rem[1:]
excel_nmd_rem = excel_nmd_rem[1:]
excel_crm_rem['Customer Contract ID'] = pd.to_numeric(excel_crm_rem['Customer Contract ID'])
excel_crm_rem['Customer Contract ID'] = excel_crm_rem['Customer Contract ID'].astype(float)
crm_remove = list(excel_crm_rem['Customer Contract ID'].drop_duplicates(keep='last'))
excel_nmd_rem['Customer Contract ID'] = pd.to_numeric(excel_nmd_rem['Customer Contract ID'])
excel_nmd_rem['Customer Contract ID'] = excel_nmd_rem['Customer Contract ID'].astype(float)
nmd_remove = list(excel_nmd_rem['Customer Contract ID'].drop_duplicates(keep='last'))





In [282]:
# removing products from log from excel
remove['ID'] = remove['ID'].astype('string')
remove = list(remove['ID'].drop_duplicates(keep='last'))



In [283]:
# dropping rows
excel_nmd2.dropna(axis = 0, how = 'all', inplace = True)
excel_nmd2.dropna(axis = 1, how = 'all', inplace = True)

excel_nmd2 = excel_nmd2[1:-1]
excel_nmd1 = excel_nmd1[1:]
excel_crm1 = excel_crm1[1:]
excel_crm2 = excel_crm2[1:]

In [284]:
# merge two sheets
excel_crm = pd.concat([excel_crm1, excel_crm2])
excel_nmd = pd.concat([excel_nmd1, excel_nmd2])

In [285]:
# reseting indexes
excel_nmd = excel_nmd.reset_index(drop=True)
excel_crm = excel_crm.reset_index(drop=True)

In [286]:
# removing contracts that are not in contract header
for index in tqdm(excel_crm.index):
    if excel_crm['Customer Contract ID'][index] not in crm_remove:
        excel_crm.drop([index], axis=0, inplace=True)


100%|██████████| 91/91 [00:00<00:00, 9549.92it/s]


In [287]:
# removing contracts that are not in contract header
for index in tqdm(excel_nmd.index):
    if excel_nmd['Customer Contract ID'][index] not in nmd_remove:
        excel_nmd.drop([index], axis=0, inplace=True)

100%|██████████| 89/89 [00:00<00:00, 61640.20it/s]


In [288]:
# mergeing two excels
excel_both = pd.concat([excel_crm, excel_nmd])


In [291]:
excel_both

,Customer Contract ID,Sales Deal #,Price Program Start Date,Price Program End Date,Price Program Name\n(free form text),Organization,Price Strategy,Tier Timeframe,Schedule Basis,Tier #,Tier Basis,Sales Lines From
30,10127.0,711069.0,2021-04-01 00:00:00,2024-03-31 00:00:00,CRM_NCOM_10127 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
31,10143.0,694090.0,2017-07-18 00:00:00,2024-04-14 00:00:00,CRM_NCOM_10143 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
32,10311.0,286461.0,2010-07-19 00:00:00,2025-12-31 00:00:00,CRM_NCOM_10311 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
33,10373.0,674334.0,2021-09-01 00:00:00,2024-04-14 00:00:00,CRM_NCOM_10373 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
34,10469.0,541582.0,2016-07-28 00:00:00,2024-04-14 00:00:00,CRM_NCOM_10469 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
83,80580.0,0000636036,2019-08-29,2024-08-16 00:00:00,SCS,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN
83,80580.0,0000679064,2019-08-29,2024-08-16 00:00:00,SCS,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN
84,80580.0,0000708557,2022-09-30,2024-08-16 00:00:00,VF,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN
85,80581.0,0000679937,2021-11-17,2024-10-31 00:00:00,SCS,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
# splitting lists of sales deals into separate rows
excel_both['Sales Deal #']= excel_both['Sales Deal #'].astype(str)
excel_both['Sales Deal #']  = excel_both['Sales Deal #'].apply(lambda x: x.split(','))
excel_both = excel_both.explode('Sales Deal #')


In [ ]:
#######PRICING########

In [68]:
# merging crm and nmd pricing
pricing_nmd.rename(columns = {'Sales Deal':'SalesDeal', 'Sales Deal Type': 'DealType', 'Sales Org': 'SalesOrg', 
'Dist.Channel':'DChan', 'Division':'Divi', 'Valid From':'ValidFr', 'Valid To':'ValidTo', 'Currency':'Curr',
'External Description':'ExDesc',  'Promotion Number':'PromotionNo','PH Level':'PLevel'}, inplace = True)
pricing_both = pd.concat([pricing_crm, pricing_nmd])
pricing_both = pricing_both.reset_index(drop=True)

In [69]:
# data types
pricing_both['PLevel']= pricing_both['PLevel'].astype('Int64')
pricing_both['Product No'] = pricing_both['PLevel'].astype(str)+'_'+pricing_both['Material/PH'].astype(str)

In [10]:
#condition
condition = (pricing_both['PLevel'] == 5) & (pricing_both['Material/PH'].str.len() == 3)
pricing_both_check= pricing_both.loc[condition]
pricing_both__wothout_check= pricing_both.loc[-condition]


In [11]:
# reset index
pricing_both_check = pricing_both_check.reset_index(drop=True)

In [12]:
# removing deleted products
for index in tqdm(pricing_both_check.index):
    if pricing_both_check['Product No'][index] in remove:
        pricing_both_check.drop([index], axis=0, inplace=True)


  0%|          | 293/165874 [00:06<1:01:53, 44.58it/s]


KeyboardInterrupt: 

In [155]:
# merging bot pricing
pricing_both_after_check = pd.concat([pricing_both_check, pricing_both__wothout_check])
pricing_both_after_check = pricing_both_after_check.reset_index(drop=True)

In [175]:
# exporting to csv
pricing_both_after_check.to_csv(f'{path}/pricinhg without removed.csv')

In [370]:
# opening csv
pricing_both_after_check = pd.read_csv(f'{path}/pricinhg without removed.csv')

/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_6869/3405298310.py:2: DtypeWarning: Columns (1,2,11,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  pricing_both_after_check = pd.read_csv(f'{path}/pricinhg without removed.csv')


In [194]:
#######PRICING AND EXCEL-MERGING

In [371]:
pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype(str)
pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].str.replace('US', '')
pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype(int)


In [372]:
pricing_both_after_check['SalesDeal']

0          284809
1          284809
2          284809
3          284809
4          284809
            ...  
1007956    263380
1007957    263380
1007958    263380
1007959    263380
1007960    263380
Name: SalesDeal, Length: 1007961, dtype: int64

In [373]:
# data types
excel_both['Sales Deal #'] = excel_both['Sales Deal #'].astype(float)
excel_both['Sales Deal #'] = excel_both['Sales Deal #'].astype(int)
#excel_both['Sales Deal #'] = excel_both['Sales Deal #'].astype(str)
#pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype('int64')
#pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype(str)


In [374]:
# merge pricing and sales
excel_both['Customer Contract ID'] = pd.to_numeric(excel_both['Customer Contract ID'])
pricing_both_after_check ['Customer'] = pd.to_numeric(pricing_both_after_check ['Customer'])
excel_both['Sales Deal #'] = pd.to_numeric(excel_both['Sales Deal #'])
pricing_both_after_check ['SalesDeal'] = pd.to_numeric(pricing_both_after_check ['SalesDeal'])

sales= pd.merge(pricing_both_after_check,excel_both, left_on='SalesDeal', right_on='Sales Deal #', how='inner')

In [ ]:
'Customer Contract ID'

In [394]:
sales

,Unnamed: 0,SeqNumber,SalesDeal,DealType,SalesOrg,DChan,Divi,Description,ValidFr,ValidTo,...,Price Program Start Date,Price Program End Date,Price Program Name\n(free form text),Organization,Price Strategy,Tier Timeframe,Schedule Basis,Tier #,Tier Basis,Sales Lines From
0,1686,13,284937,ZC01,US50,0,0,CRM-LOCAL,7/19/2010,12/31/9999,...,2010-07-19,2025-12-31 00:00:00,CRM_NCOM_12007 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
1,1687,13,284937,ZC01,US50,0,0,CRM-LOCAL,7/19/2010,12/31/9999,...,2010-07-19,2025-12-31 00:00:00,CRM_NCOM_12007 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
2,1688,13,284937,ZC01,US50,0,0,CRM-LOCAL,7/19/2010,12/31/9999,...,2010-07-19,2025-12-31 00:00:00,CRM_NCOM_12007 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
3,1689,13,284937,ZC01,US50,0,0,CRM-LOCAL,7/19/2010,12/31/9999,...,2010-07-19,2025-12-31 00:00:00,CRM_NCOM_12007 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
4,1690,13,284937,ZC01,US50,0,0,CRM-LOCAL,7/19/2010,12/31/9999,...,2010-07-19,2025-12-31 00:00:00,CRM_NCOM_12007 - CRMGENERAL,Cardiac Rhythm Management,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8522,1005507,*068,726738,ZC01,US10,0,0,Alvarado Hospital LLC Pricing,01/03/2023,31/12/2035,...,2023-03-01,2029-02-27 00:00:00,SCS,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN
8523,1005508,*068,726738,ZC01,US10,0,0,Alvarado Hospital LLC Pricing,01/03/2023,31/12/2035,...,2023-03-01,2029-02-27 00:00:00,SCS,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN
8524,1005509,*068,726738,ZC01,US10,0,0,Alvarado Hospital LLC Pricing,01/03/2023,31/12/2035,...,2023-03-01,2029-02-27 00:00:00,SCS,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN
8525,1005510,*068,726738,ZC01,US10,0,0,Alvarado Hospital LLC Pricing,01/03/2023,31/12/2035,...,2023-03-01,2029-02-27 00:00:00,SCS,Neuromodulation,NaN,NaN,NaN,NaN,NaN,NaN


In [395]:
# aggregating  by contract id
sales_gr = sales.groupby([
  #  'SalesDeal',
'Customer Contract ID',
'Price Program Name\n(free form text)'
]).size().reset_index(name='counts')

In [385]:
#####XML######
# Getting numbers and list with products
list_all = []
Contract = tree.xpath('.//ContractOffer')
for contract in Contract:
    Program = contract.xpath('.//Program')
    for program in Program:
        Discount = program.xpath('.//DiscountOffListLI')
        for discount in Discount:
            Price = discount.xpath('.//LIPrices')
            for price in Price:
                list_all.append([contract.attrib['Customer'], 
                program.attrib['CustomerContractId'], program.attrib['SalesDealNo'], program.attrib['Name'],
                discount.attrib['ProductNum'],
                price.attrib['PriceHigh']])
# dataframes
xml_all = pd.DataFrame(list_all)
xml_all.columns =['Customer ID', 'Contract ID','DealNo',
'Price Program Name', 
'Product ID', 'Price']

              


In [386]:
# splitting lists of deals to separate rows
xml_all['DealNo']= xml_all['DealNo'].astype(str)
xml_all['DealNo']  = xml_all['DealNo'].apply(lambda x: x.split(','))
xml_all = xml_all.explode('DealNo')


In [389]:
# data types
xml_all['DealNo'] = pd.to_numeric(xml_all['DealNo'])
#xml_all['DealNo'] = xml_all['DealNo'].astype(str)

In [387]:
xml_all = xml_all.drop_duplicates(subset = ['Product ID', 'Price', 'Price Program Name','Customer ID'],keep = 'first').reset_index(drop = True)

In [390]:
# jak usune duplikaty to moge porownywac ceny w oparciu o customer, bez sales deal

,Customer ID,Contract ID,DealNo,Price Program Name,Product ID,Price
0,0000000019,61129,511417,SCS,4_8502,1000.00|USD
1,0000000019,61129,511417,SCS,4_8508,1000.00|USD
2,0000000019,61129,511417,SCS,5_12013,1.00|USD
3,0000000019,61129,511417,SCS,5_12016,1.00|USD
4,0000000019,61129,511417,SCS,5_12017,1.00|USD
...,...,...,...,...,...,...
6816,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_L331,2910.0|USD
6817,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_M301,4200.0|USD
6818,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_PLDS,485.0|USD
6819,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_U225,5820.0|USD


In [391]:
# data types
xml_all['Contract ID'] = pd.to_numeric(xml_all['Contract ID'])
xml_all['Price'] = xml_all.Price.str.extract(r"(\d+\.\d+)")
xml_all['Price'] = xml_all['Price'].astype(float)

In [392]:
# aggregating xml
xml_all_gr = xml_all.groupby([
    # 'DealNo', 
    'Contract ID', 'Price Program Name'
]).size().reset_index(name='counts')

In [393]:
xml_all

,Customer ID,Contract ID,DealNo,Price Program Name,Product ID,Price
0,0000000019,61129,511417,SCS,4_8502,1000.0
1,0000000019,61129,511417,SCS,4_8508,1000.0
2,0000000019,61129,511417,SCS,5_12013,1.0
3,0000000019,61129,511417,SCS,5_12016,1.0
4,0000000019,61129,511417,SCS,5_12017,1.0
...,...,...,...,...,...,...
6816,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_L331,2910.0
6817,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_M301,4200.0
6818,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_PLDS,485.0
6819,0000013167,13167,673049,CRM_NCOM_13167 - CRMGENERAL,5_U225,5820.0


In [396]:
# Merging dataframes- counts
xml_all_gr['Contract ID'] = pd.to_numeric(xml_all_gr['Contract ID'])
sales_gr['Customer Contract ID'] = pd.to_numeric(sales_gr['Customer Contract ID'])
xml_all_gr['DealNo'] = pd.to_numeric(xml_all_gr['DealNo'])
sales_gr['SalesDeal'] = pd.to_numeric(sales_gr['SalesDeal'])

df_merged = pd.merge(sales_gr, xml_all_gr,  how='outer', left_on=['Customer Contract ID', 'Price Program Name\n(free form text)'], right_on = ['Contract ID', 'Price Program Name'])
df_merged.rename(columns = {'counts_x':'count from excel', 'counts_y':'count from xml'}, inplace = True)

In [397]:
# checking if the same
df_merged['Are the same?'] = ''
for index in tqdm(df_merged.index):
    if df_merged['count from xml'][index] == df_merged['count from excel'][index]:
        df_merged['Are the same?'][index] = 'yes'
    else:
        df_merged['Are the same?'][index] = 'no'
df_merged.to_excel(f'{path}/result counts5.xlsx')

  0%|          | 0/227 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_6869/896799660.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Are the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_6869/896799660.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Are the same?'][index] = 'no'
100%|██████████| 227/227 [00:00<00:00, 12418.73it/s]


In [79]:
sales = sales.iloc[:,:-6]

In [80]:
xml_all['Contract ID'] = pd.to_numeric(xml_all['Contract ID'])
sales['Customer'] = pd.to_numeric(sales['Customer'])

In [ ]:
########################################################################

In [81]:
df_merged_extended = pd.merge(sales, xml_all,  how='outer', left_on=['Customer', 'Price Program Name\n(free form text)',
'Product No'
], 
right_on = ['Contract ID', 'Price Program Name',
'Product ID'
])

In [82]:

df_merged_extended['Price'] = pd.to_numeric(df_merged_extended['Price'])
df_merged_extended['Rate'] = pd.to_numeric(df_merged_extended['Rate'])

In [83]:
df_merged_extended['Are prices the same?'] = ''

for index in tqdm(df_merged_extended.index):
    if df_merged_extended['Price'][index] == df_merged_extended['Rate'][index]:
        df_merged_extended['Are prices the same?'][index] = 'yes'
    else:
        df_merged_extended['Are prices the same?'][index] = 'no'
    



  0%|          | 0/15522 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1396/1648672751.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are prices the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1396/1648672751.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are prices the same?'][index] = 'no'
100%|██████████| 15522/15522 [00:01<00:00, 10832.27it/s]


In [49]:
df_merged_extended.columns

Index(['Unnamed: 0', 'SeqNumber', 'SalesDeal', 'DealType', 'SalesOrg', 'DChan',
       'Divi', 'Description', 'ValidFr', 'ValidTo', 'Curr', 'ExDesc',
       'PromotionNo', 'CondType', 'Customer', 'PLevel', 'Material/PH', 'Rate',
       'Per', 'Unit', 'Del', 'CustName', 'Customer Name', 'Product Line',
       'Product No', 'Customer Contract ID', 'Sales Deal #',
       'Price Program Start Date', 'Price Program End Date',
       'Price Program Name\n(free form text)', 'Organization', 'Customer ID',
       'Contract ID', 'DealNo', 'Price Program Name', 'Product ID', 'Price',
       'Are prices the same?'],
      dtype='object')

In [84]:

df_merged_extended.to_excel(f'{path}/result prices.xlsx')